In [ ]:
# --- INITIALIZATION: IMPORTING NECESSARY LIBRARIES ---
# Fundamental libraries for data manipulation and numerical computation.
import pandas as pd
import numpy as np
# Libraries for data visualization.
import matplotlib.pyplot as plt
import seaborn as sns
# Scikit-learn modules for preprocessing, data splitting, and model evaluation.
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, f1_score, precision_recall_curve, auc
# Statsmodels provides detailed statistical output for feature selection.
import statsmodels.api as sm

# --- PHASE 1: EXPLORATORY DATA ANALYSIS AND CLEANSING ---

# Load the dataset from the source file.
df = pd.read_csv('goodreads_books_clean.csv')

# Conduct an initial inspection to understand data structure and integrity.
print("Dataset Dimensionality:", df.shape)
print("\nData Types and Null Counts:")
print(df.info())
print("\nSummary Statistics for Numerical Features:")
print(df.describe())

# Quantify and visualize the presence of missing data.
plt.figure(figsize=(10,6))
sns.heatmap(df.isnull(), cbar=False, yticklabels=False)
plt.title('Heatmap of Missing Values')
plt.show()

# Implement data cleansing procedures.
# For numerical fields with missing data, imputation with the median is robust to outliers.
df['num_pages'].fillna(df['num_pages'].median(), inplace=True)
# For categorical fields, imputation with the most frequent category is appropriate.
df['language_code'].fillna(df['language_code'].mode()[0], inplace=True)

# Engineer the binary target variable based on a percentile threshold.
bestseller_threshold = df['ratings_count'].quantile(0.8)
df['is_bestseller'] = (df['ratings_count'] >= bestseller_threshold).astype(int)
print(f"Target Variable Distribution:\n{df['is_bestseller'].value_counts(normalize=True)}")

# Visualize the class balance of the newly created target variable.
sns.countplot(x='is_bestseller', data=df)
plt.title('Class Distribution of Bestseller Status')
plt.xlabel('Bestseller Status (0 = No, 1 = Yes)')
plt.ylabel('Count')
plt.show()

# --- PHASE 2: FEATURE SELECTION AND PREPROCESSING ---

# Define the initial set of candidate features for the model.
feature_columns = ['average_rating', 'num_pages', 'ratings_count', 'language_code']
X = df[feature_columns]
y = df['is_bestseller']

# Preprocess the features: encode categorical variables and scale numerical ones.
X_encoded = pd.get_dummies(X, columns=['language_code'], prefix='lang', drop_first=True)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_encoded)
# Convert back to DataFrame for clarity in subsequent steps.
X_scaled = pd.DataFrame(X_scaled, columns=X_encoded.columns)

# Partition the data into training and testing sets, preserving the class distribution.
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

# Employ statistical backward elimination for feature selection.
# Adding a constant term for the intercept in the statistical model.
X_train_sm = sm.add_constant(X_train)
logit_model = sm.Logit(y_train, X_train_sm).fit()
print(logit_model.summary())
# Based on the p-values in the summary, features with p > 0.05 will be iteratively removed.
# Assume the final selected features are: 'average_rating', 'ratings_count', 'lang_eng'
selected_features = ['average_rating', 'ratings_count', 'lang_eng']
X_train_final = X_train[selected_features]
X_test_final = X_test[selected_features]

# --- PHASE 3: MODEL IMPLEMENTATION FROM SCRATCH ---

def sigmoid(z):
    """Computes the sigmoid function, mapping input z to a probability between 0 and 1."""
    return 1 / (1 + np.exp(-z))

def compute_cost(y_true, y_pred_prob):
    """Computes the binary cross-entropy loss.
    Clipping is applied to prevent numerical instability from log(0)."""
    y_pred_clipped = np.clip(y_pred_prob, 1e-15, 1 - 1e-15)
    return -np.mean(y_true * np.log(y_pred_clipped) + (1 - y_true) * np.log(1 - y_pred_clipped))

def gradient_descent(X, y_true, learning_rate=0.1, iterations=1000):
    """Performs gradient descent to learn the optimal model parameters."""
    m, n = X.shape
    weights = np.zeros(n)
    bias = 0
    cost_history = []

    for i in range(iterations):
        # Forward pass: compute linear combination and predicted probabilities.
        linear_model = np.dot(X, weights) + bias
        y_pred_prob = sigmoid(linear_model)
        
        # Compute and store the cost for monitoring convergence.
        cost = compute_cost(y_true, y_pred_prob)
        cost_history.append(cost)
        
        # Backward pass: compute gradients for weights and bias.
        gradient_w = (1 / m) * np.dot(X.T, (y_pred_prob - y_true))
        gradient_b = (1 / m) * np.sum(y_pred_prob - y_true)
        
        # Update model parameters.
        weights -= learning_rate * gradient_w
        bias -= learning_rate * gradient_b
        
        # Monitor convergence every 100 iterations.
        if i % 100 == 0:
            print(f"Iteration {i}: Cost = {cost:.4f}")
        # Early stopping condition based on minimal cost improvement.
        if i > 10 and abs(cost_history[-2] - cost_history[-1]) < 1e-7:
            print(f"Convergence achieved at iteration {i}.")
            break
            
    return weights, bias, cost_history

# Execute the custom training algorithm.
trained_weights, trained_bias, historical_cost = gradient_descent(X_train_final.values, y_train.values)

# Visualize the convergence of the gradient descent algorithm.
plt.plot(historical_cost)
plt.title('Convergence Profile of Gradient Descent')
plt.xlabel('Iteration')
plt.ylabel('Log Loss')
plt.grid(True)
plt.show()

# --- PHASE 4: MODEL PREDICTION AND EVALUATION ---

def predict_classes(X, weights, bias, decision_threshold=0.5):
    """Generates binary class predictions based on learned parameters and a threshold."""
    linear_combination = np.dot(X, weights) + bias
    probabilities = sigmoid(linear_combination)
    return (probabilities >= decision_threshold).astype(int), probabilities

# Generate predictions for the test set.
y_pred_binary, y_pred_probability = predict_classes(X_test_final.values, trained_weights, trained_bias)

# Conduct a comprehensive model evaluation.
print("\n" + "="*50)
print("MODEL PERFORMANCE EVALUATION")
print("="*50)
print(f"F1-Score: {f1_score(y_test, y_pred_binary):.4f}\n")

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_binary))
print("\nDetailed Classification Report:")
print(classification_report(y_test, y_pred_binary))

# Generate and plot the Precision-Recall curve.
precision, recall, _ = precision_recall_curve(y_test, y_pred_probability)
pr_auc = auc(recall, precision)

plt.figure()
plt.plot(recall, precision, label=f'Custom Logistic Model (AUC = {pr_auc:.3f})')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend()
plt.grid(True)
plt.show()
